Imports:

In [ ]:
import numpy as np
import jax
import jax.numpy as jnp
from jax.example_libraries import stax, optimizers
import matplotlib.pyplot as plt
import pyvista as pv
import pinns 
import datetime
import jax.scipy.optimize
import jax.flatten_util
import scipy
import scipy.optimize
import random

rnd_key = jax.random.PRNGKey(1234)
np.random.seed(14124)

Set the default precision and the execution device.

In [ ]:
jax.config.update("jax_enable_x64", False)
print("GPU devices: ", jax.devices('gpu'))
dev = jax.devices('gpu')[0] if len(jax.devices('gpu'))>0 else jax.devices('cpu')[0]

### Geometry definition 

Define the geometry patches:

In [ ]:
def get_wheel(angle: float, n: int, r0: float, r1: float, R0: float, R1: float, h: float, H: float):

    basis1 = pinns.functions.BSplineBasisJAX(np.array([-1, 0, 1]), 2)
    basis2 = pinns.functions.BSplineBasisJAX(np.array([-1, 1]), 2)
    basis3 = pinns.functions.BSplineBasisJAX(np.array([-1, 1]), 1)

    def tmp_gen(angle, r_0, r_1):
        pts = np.zeros([4, 3, 2, 3])
        weights = np.ones([4, 3, 2])

        a = np.pi/2-angle/2
        rs = np.linspace(r_0, r_1, 4)
        pts[-1, 0, 0, :] = [np.cos(-angle/2), np.sin(-angle/2), 0]
        pts[-1, 1, 0, :] = [1/np.sin(a), 0, 0]
        pts[-1, 2, 0, :] = [np.cos(angle/2), np.sin(angle/2), 0]
        pts[0, :, 0, :2] = rs[0] * pts[-1, :, 0, :2]
        pts[1, :, 0, :2] = rs[1] * pts[-1, :, 0, :2]
        pts[2, :, 0, :2] = rs[2] * pts[-1, :, 0, :2]
        pts[3, :, 0, :2] = rs[3] * pts[-1, :, 0, :2]
        pts[0, :, 0, 2] = -1
        pts[1, :, 0, 2] = -1
        pts[2, :, 0, 2] = -1
        pts[3, :, 0, 2] = -1
        pts[:, :, 1, :] = pts[:, :, 0, :]
        pts[:, :, 1, 2] = -pts[:, :, 1, 2]
        weights[:, 1, :] = np.sin(a)

        return pts, weights

    geoms = dict()

    pts, weights = tmp_gen(angle, r0, r1)
    pts[:, :, :, 2] *= H/2
    # pts[2:,:,:,2] *= h/2
    pts[3, 1, :, 0] = pts[3, 0, :, 0]

    geoms['in_flat'] = pinns.geometry.PatchNURBSParam(
        [basis1, basis2, basis3], pts, weights, 0, 3, 0)
    geoms['in_flat'].rotate((0, 0, angle/2))

    pts, weights = tmp_gen(angle, R0, R1)
    pts[:2, :, :, 2] *= h/2
    pts[2:, :, :, 2] *= H/2
    pts[0, 1, :, 0] = pts[0, 0, :, 0]
    geoms['out_flat'] = pinns.geometry.PatchNURBSParam(
        [basis1, basis2, basis3], pts, weights, 0, 3, 0)
    geoms['out_flat'].rotate((0, 0, angle/2))

    
        
    angle_straight = (2*np.pi/n if n >=4 else np.pi/2) - angle

        
    pts, weights = tmp_gen(angle_straight, r0, r1)
    pts[:, :, :, 2] *= H/2
    # pts[2:,:,:,2] *= h/2
    geoms['in_round'] = pinns.geometry.PatchNURBSParam(
        [basis1, basis2, basis3], pts, weights, 0, 3, 0)
    geoms['in_round'].rotate((0, 0, angle_straight/2+angle))

    pts, weights = tmp_gen(angle_straight, R0, R1)
    pts[:2, :, :, 2] *= h/2
    pts[2:, :, :, 2] *= H/2
    geoms['out_round'] = pinns.geometry.PatchNURBSParam(
        [basis1, basis2, basis3], pts, weights, 0, 3, 0)
    geoms['out_round'].rotate((0, 0, angle_straight/2+angle))

    basis1 = pinns.functions.BSplineBasisJAX(
        np.array([-1, -0.5, 0, 0.5, 1]), 2)
    basis2 = pinns.functions.BSplineBasisJAX(np.array([-1, 1]), 2)
    basis3 = pinns.functions.BSplineBasisJAX(np.array([-1, 1]), 2)

    weights = np.ones([6, 3, 3])
    pts = np.zeros([6, 3, 3, 3])

    pts1, _ = tmp_gen(angle, r0, r1)
    pts1[3, 1, :, 0] = pts1[3, 0, :, 0]
    pts[0, :, 0, :] = pts1[-1, :, 0, :]
    pts[0, :, -1, :] = pts1[-1, :, 1, :]
    pts[0, :, 1, :] = 0.5*(pts[0, :, 0, :]+pts[0, :, -1, :])

    pts2, _ = tmp_gen(angle, R0, R1)
    pts2[0, 1, :, 0] = pts2[0, 0, :, 0]
    pts[-1, :, 0, :] = pts2[0, :, 0, :]
    pts[-1, :, -1, :] = pts2[0, :, 1, :]
    pts[-1, :, 1, :] = 0.5*(pts[-1, :, 0, :]+pts[-1, :, -1, :])

    linsp = np.linspace(0, 1, basis1.n)

    pts[-1, :, :, 2] *= h/2
    pts[0, :, :, 2] *= H/2

    for i in range(1, basis1.n-1):
        pts[i, :, :, 2] = (1-linsp[i]**0.25)*pts[0, :, :, 2] + \
            linsp[i]**0.25*pts[-1, :, :, 2]
        pts[i, :, :, 0] = (1-linsp[i])*pts[0, :, :, 0] + \
            linsp[i]*pts[-1, :, :, 0]
        pts[i, :, :, 1] = (1-linsp[i]**4)*pts[0, :, :, 1] + \
            linsp[i]**4*pts[-1, :, :, 1]

    geoms['spoke'] = pinns.geometry.PatchNURBSParam(
        [basis1, basis2, basis3], pts, weights, 0, 3, 0)
    geoms['spoke'].rotate((0, 0, angle/2))
    
    geoms_all = dict()
    for i in range(n):
        alpha = i*(2*np.pi/n if n >=4 else np.pi/2)

        for k in geoms:
            geoms_all[k+'_'+str(i)] = pinns.geometry.PatchNURBSParam(
                geoms[k].basis, geoms[k].knots(), geoms[k].weights(), 0, 3, 0)
            geoms_all[k+'_'+str(i)].rotate((0, 0, alpha))
    return geoms_all


geoms = get_wheel(np.pi/8, 2, 0.1, 0.2, 1, 1.2, 0.1, 0.25)
names = list(geoms.keys())


Export as a VTK file for visualization in paraview.

In [ ]:
# with jax.disable_jit():
#     objects = [pinns.extras.plot(geoms[n], dict(), N=32) for n in geoms]
# 
# obj = objects[0]
# for i in range(1, len(objects)):
#     obj = obj.merge(objects[i])
# 
# obj.save('testing.vtk')

Determine the connectivity of the patches:

In [ ]:
with jax.disable_jit(True):
    connectivity = pinns.geometry.match_patches(geoms, verbose=False)

for c in connectivity:
    print(c)

### ANN spaces definition

The network is an MLP with residual connections and width set by the `nl` paraemter.
There are 2 spaces defined: first has 0 Dirichlet BCs on one facet and the other has no Dirichlet BCs enforced. 

In [ ]:
nl = 20
acti =  stax.elementwise(lambda x: jax.nn.leaky_relu(x)**2)
w_init = jax.nn.initializers.normal()

block_first = stax.serial(stax.FanOut(2),stax.parallel(stax.serial(stax.Dense(nl,W_init = w_init), acti, stax.Dense(nl,W_init = w_init), acti),stax.Dense(nl,W_init = w_init)),stax.FanInSum)
block = stax.serial(stax.FanOut(2),stax.parallel(stax.serial(stax.Dense(nl,W_init = w_init), acti, stax.Dense(nl,W_init = w_init), acti),stax.Dense(nl,W_init = w_init)),stax.FanInSum)
nn = stax.serial(block_first,block, stax.Dense(3))

space_bc = pinns.FunctionSpaceNN(pinns.DirichletMask(nn, 3, [(-1,1), (-1,1), (-1,1)], [{'dim': 0, 'end': 0}]), ((-1,1), (-1,1), (-1,1))) 
space = pinns.FunctionSpaceNN(nn,((-1,1), (-1,1), (-1,1)))

### PINNs

Define the PINN class. The loss has to be defined. In this case, the nonlinear geometry is used.

In [ ]:
class Pinn(pinns.PINN):
    
    def __init__(self):
          
        self.weights = {n: space_bc.init_weights(rnd_key) if 'in' in n else space.init_weights(rnd_key) for n in names}
        self.solutions = pinns.connectivity_to_interfaces({n: space_bc if 'in' in n else space for n in names}, connectivity)
        
        E = 0.02e5
        nu = 0.1
        self.E = E
        self.nu = nu
        
        self.lamda = E*nu/(1+nu)/(1-2*nu)
        self.mu = E/2/(1+nu)

        rho = 0.02
        g = 9.81
        self.rho = rho
        
        self.f = np.array([0,-g*rho,0]) 
        self.energy = lambda F,C,J,params: params[0]*jnp.sum(F**2, axis=(-2,-1)) + params[1]*jnp.abs(J)**2*jnp.sum(jnp.linalg.inv(F)**2, axis=(-1,-2)) + params[2]*J**2 - params[3]*jnp.log(jnp.abs(J))+params[4]
        self.energy = lambda F,C,J,params: 0.5*self.mu*(C[...,0,0]+C[...,1,1]+C[...,2,2]-3)-self.mu*jnp.log(jnp.abs(J))+0.5*self.lamda*jnp.log(jnp.abs(J))**2
        
        self.a = 0.5*self.mu
        self.b = 0.0
        self.c = 0.0
        self.d = self.mu
        self.e = -1.5*self.mu

        super(Pinn, self).__init__(geoms)
   
 
    def loss(self, training_parameters, points):
        

        jacs = [pinns.functions.jacobian(lambda x : self.solutions[n](training_parameters, x))(points[n].points_reference) for n in names]
        jacs_x = [points[names[i]].jacobian_transformation(jacs[i]) for i in range(len(names))]
        Fs = [jnp.eye(3)+jacs_x[i] for i in range(len(names))]
        Cs = [jnp.einsum('mij,mik->mjk', Fs[i], Fs[i]) for i in range(len(names))]
        
        dets = [jnp.linalg.det(Fs[i]) for i in range(len(names))]
         
        Es = [jnp.dot(self.energy(Fs[i], Cs[i], dets[i], [self.a, self.b,self.c,self.d,self.e]), points[names[i]].dx()) for i in range(len(names))]
        rhss = [jnp.dot(dets[i] * jnp.einsum('k,mk->m', self.f, self.solutions[names[i]](training_parameters, points[names[i]].points_reference)), points[names[i]].dx()) for i in range(len(names))] 

        return sum(Es) - sum(rhss)
    
        
model = Pinn()  

### Training


In [ ]:
opt_type = 'ADAM'

if opt_type == 'ADAM':
    
    
    n_batches = 500 
    n_points = 1000000
    n_points_batch = n_points//n_batches
    
    lr_opti = optimizers.piecewise_constant([2000,3000,4000,5000,7000], [0.005, 0.005/2, 0.005/4, 0.005/8,0.005/16,0.005/32])
    #lr_opti = optimizers.piecewise_constant([2000,3000,4000,5000], [0.005, 0.005/4, 0.005/4,0.005/4,0.005/4])
    # lr_opti = optimizers.piecewise_constant([7000], [0.01/2, 0.001])
    opt_init, opt_update, get_params = optimizers.adam(lr_opti)

    opt_state = opt_init(model.weights)
    weights_init = model.weights
    
    # get initial parameters
    params = get_params(opt_state)

    loss_grad = jax.jit(lambda ws, pts: (model.loss(ws, pts), jax.grad(model.loss)(ws, pts)), device = dev)

    def step(params, opt_state, key):
        # points = model.get_points_MC(5000)
        points = model.points_MonteCarlo(n_points_batch, key)
        loss = model.loss(params, points)
        grads = jax.grad(model.loss)(params, points)
        #loss, grads = loss_grad(params, points)
        opt_state = opt_update(0, grads, opt_state)

        params = get_params(opt_state)
        
        return params, opt_state, loss

    step_compiled = jax.jit(step, device = dev)
    step_compiled(params, opt_state, rnd_key)

    n_epochs = 20

    hist = []
    hist_weights = []
    
    # min_loss = 10000
    tme = datetime.datetime.now()
    for ep in range(n_epochs):   
        
        losses = []
        for b in random.sample(range(n_batches), n_batches):
         
            params, opt_state, loss = step_compiled(params, opt_state, jax.random.PRNGKey(b+0*np.random.randint(1000)))
            #print("\tbatch %d/%d"%(b+1, n_batches))
            hist.append(loss)
            losses.append(loss)
        
        hist_weights.append(params.copy())
        print('Epoch %d/%d - mean loss %e, max loss %e, min loss %e, std loss %e'%(ep+1, n_epochs, np.mean(losses), np.max(losses), np.min(losses), np.std(losses)))
        
    # update params
    model.weights = params
    weights = params
    tme = datetime.datetime.now() - tme
    print('Elapsed time ', tme)


### Plot 

Save the solution as a `.vtk` file.

In [ ]:
with jax.disable_jit():
    pv_objects = [pinns.extras.plot(geoms[n], {'displacement': lambda y: model.solutions[n](weights, y)}, N= 25) for n in geoms]

obj = pv_objects[0]
for i in range(1,len(pv_objects)):
    obj = obj.merge(pv_objects[i])
obj.save('solution_wheel.vtk')

In [ ]:
for ep in range(n_epochs):
    with jax.disable_jit():
        pv_objects = [pinns.extras.plot(geoms[n], {'displacement': lambda y: model.solutions[n](hist_weights[ep], y)}, N= 25) for n in geoms]

    obj = pv_objects[0]
    for i in range(1,len(pv_objects)):
        obj = obj.merge(pv_objects[i])
    obj.save('solution_wheel_epoch_%d.vtk'%(ep+1))